In [1]:
import json
from src.data_processing import print_progress

def ranked_vectors(path):
    subs = []
    for index in range(1780):
        print_progress(index, 1780)
        full_path = "results/{}/{}".format(path, index)
        with open(full_path) as f:
            subs.append(json.load(f))
    return subs

In [2]:
def number_of_substitutions(index, subs):
    tot_sum = 0
    for k, v in subs[index].items():
        tot_sum += len(v)
    return tot_sum

In [3]:
def generate_ranked_vector(index, mvl, subs):
    vec = [0] * mvl
    scores = []
    for k, v in subs[index].items():
        scores.extend([score[1] for score in v])
    vec[:len(scores)] = list(sorted(scores, reverse=True))
    
    return vec

In [4]:
import numpy as np

Load the vectors, dump as binary

In [6]:
locations = (['phonetic_filter_no_pos',
              'phonetic_filter_with_pos',
              'all_trigram_no_pos',
              'all_trigram_with_pos'])

In [7]:
max_columns = 10000
for p in locations:
    
    subs = ranked_vectors(p)
    max_vector_length = (min(max(number_of_substitutions(i, subs) 
                             for i in range(1780)),
                             max_columns)) 
    
    vectors = ([generate_ranked_vector(i, max_vector_length, subs) 
                for i in range(1780)])
    X = np.array(vectors)
    with open('vectors/{}.np'.format(p), 'wb') as f:
        np.save(f, X)

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 99.9% 

In [8]:
from src.data_processing import load_data
task1, task2, task3, min_pairs, strings, pun_strings = load_data()

In [9]:
y = np.array([int(context['pun']) for context in task1])

In [68]:
from sklearn.model_selection import KFold

In [69]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)

10

In [10]:
from sklearn import svm

In [ ]:

columns = 10
from sklearn.metrics import classification_report
for train_index, test_index in kf.split(X):
    
    print(train_index)
    X_train, X_test = X[train_index, :columns], X[test_index, :columns]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = svm.SVC(kernel='linear')
    clf.fit(X_train, y_train)
    print(classification_report(y_test, clf.predict(X_test)))
    input()
    

[ 178  179  180 ..., 1777 1778 1779]


In [ ]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', verbose=True)
scores = cross_val_score(clf, X, y, cv=3)

[LibSVM]

In [23]:
scores

array([ 0.67171717,  0.41582492,  0.61317568])